In [2]:
import psycopg2
import pandas as pd
import numpy as np
import re
import datetime as dt

In [1285]:
df = pd.read_csv('Reformation_table.csv')

In [1286]:
df.rename({'brand' : 'brand_name'}, axis=1, inplace=True)
df.rename({'product_name' : 'display_name'}, axis=1, inplace=True)
df.rename({'url' : 'product_url'}, axis=1, inplace=True)
df.rename({'available' : 'size'}, axis=1, inplace=True)

In [1287]:
df.color = df.color.str.replace(']',"").str.replace('[',"").str.replace("'","").str.split(', ')

In [1288]:
df.image = df.image.str.replace(']',"").str.replace('[',"").str.replace("'","").str.split(', ')

In [1289]:
GranCat = pd.read_csv("LowLevelCatagoryRegex.csv", header=None, skiprows=[0])

def process_cat(df):
    # get all the values in category regex
    subCats = GranCat.iloc[:, 1:]
    Cats = []
    for index, row in subCats.iterrows():
        for i in GranCat.columns[1:]:
            if pd.isnull(row[i]): continue
            else: Cats.append(row[i])
               
    # assign potential category for each clothing row based on title, else other
    clothesCat = []
    for index, row in df.iterrows():
        rowCat = float("NaN")
        for cat in Cats:
            if re.search(cat, row['display_name'].lower()):
                rowCat = cat
                print(rowCat)
                break
        if pd.isnull(rowCat):
            clothesCat.append('other')
        else:
            clothesCat.append(rowCat)

    return clothesCat

# set parent category to each product
def get_cat(x):
    if x == 'other': return x
    else: return [GranCat[0][i] for i in GranCat.index if x in GranCat.iloc[i].to_list()][0]

# get low_level column given a df
def get_lowlevel(df):
    df['low_level_new'] = process_cat(df)
    df['low_level_new'] = df['low_level_new'].apply(lambda x: get_cat(x))
    return df
# get unique id
def get_id(df, date='20210901'):
    brand = df['brand_name'][0]
    clothing_ids = []
    for i in range(len(df.index)):
        clothing_id = date + brand + str(i)
        clothing_ids.append(clothing_id)
    return clothing_ids
def get_dic_image(row):
    color_image_dict = {'color': 'image_url'}
    for i in range(len(row['color'])):
        #print(i)
        #{row['color'][i]: row['image'][i]}
        try:
            color_image_dict = {row['color'][i]:row['image'][i]}
        except:
             color_image_dict = {row['color'][i]:row['image'][0]}
            

        
    return color_image_dict

def get_price(x): 
    try: return x.split(': ')[1]
    except: return 'Unknown'

def change_gender(x):
    if "Men" in x and "women" not in x.lower(): return 'Men'
    else: return 'Women'

In [1290]:
##### get mapped_id
GranRegs = pd.read_csv('MaterialProxy.csv', header=None, skiprows=[0])
GranRegs = GranRegs.drop(columns=[1])
GranRegs.columns = list(range(len(GranRegs.columns)))

In [1291]:
df['fabric'] = df['fabric'].fillna('')

In [1292]:
#extract materials

def extractMaterials(df):
    fabric_text = df['fabric']
    fabric_text = str(fabric_text).replace('solvent', '').replace('Grown', '').replace('grown', '').replace('organically', 'organic').replace('Organically', 'organic')
    splitted_fabric = fabric_text.replace("'", '').replace("[", '').replace("]", '').replace(",", '').replace('’', '').replace('.', '').replace('–', '').replace('-', '').split()
    pattern = "\d{1,3}[$%]"
    
    description_text = df['description'] + df['display_name'] + df['fabric']
    description_text = str(description_text).replace('solvent', '').replace('Grown', '').replace('grown', '').replace('organically', 'organic').replace('Organically', 'organic')
    splitted_description = description_text.replace("'", '').replace("[", '').replace("]", '').replace(",", '').replace('’', '').replace('.', '').replace('–', '').replace('-', '')
    pattern = "\d{1,3}[$%]"
    
    mat_list = []
    extra_materials = []
    
    
    ## getting the materials regex matching
    subRegs = GranRegs.iloc[:, 1:]
    Regs = []
    materialsReg = []
    for index, row in subRegs.iterrows():
        for i in range(1,len(GranRegs.columns)):
            if pd.isnull(row[i]): continue
            else: Regs.append(row[i])
    total_percentage = [] 
    is_no_percentage = True 
    for i in range(len(splitted_fabric)):
        if re.fullmatch(pattern,splitted_fabric[i]):
            total_percentage.append(float(splitted_fabric[i].replace("%", '')))
            if sum(total_percentage) <= 100:
            
                if splitted_fabric[i+1].lower() == 'organic':
                    mat_list.append(splitted_fabric[i] + ' ' + splitted_fabric[i+1].lower() + ' ' + splitted_fabric[i+2].lower())
                    is_no_percentage = False
                    
                elif splitted_fabric[i+1].lower() == 'recycled':
                    mat_list.append(splitted_fabric[i] + ' ' + splitted_fabric[i+1].lower() + ' ' + splitted_fabric[i+2].lower())
                    is_no_percentage = False
                
                elif splitted_fabric[i+1].lower() == 'tencel™':
                    mat_list.append(splitted_fabric[i] + ' ' + splitted_fabric[i+1].lower() + ' ' + splitted_fabric[i+2].lower())
                    is_no_percentage = False
                    
                else:
                    if splitted_fabric[i] + ' ' + splitted_fabric[i+1].lower() not in mat_list:
                        mat_list.append(splitted_fabric[i] + ' ' + splitted_fabric[i+1].lower())
                        is_no_percentage = False
                        
    if sum(total_percentage) < 100 or is_no_percentage == True:
        for reg in Regs:
            #print(splitted_description.lower())
            if re.search(reg, splitted_description.lower()):
                match = re.search(reg, splitted_description.lower())
                matched_material = splitted_description[match.start():match.end()].lower()
                
                if matched_material not in extra_materials:
                    extra_materials.append(matched_material)
    
    #print(extra_materials)
    if len(extra_materials)>1:
        if "cotton" in extra_materials and "organic cotton" in extra_materials:
            mat_list.append("100% organic cotton")
        if "leather" in extra_materials:
            mat_list.append("100% leather")
    elif len(extra_materials)!=0:
        mat_list.append("100% " + extra_materials[0])

            
    
    return mat_list



In [1293]:
df['materials'] = df.apply(lambda x: extractMaterials(x), axis=1)

In [1294]:
df[df["materials"].str.len() == 0]['fabric']

52                            
64          ['Dry clean only']
88          ['Dry clean only']
138    ['Wash cold\nLine dry']
Name: fabric, dtype: object

In [1295]:
#df.insert(loc=0, column='clothing_id', value=get_id(df,'20210825'))
#df['material_dict'] = ''
#df['overallscore'] = 4.5
df['color'] = df['color'].fillna('Unknown')
#df = df.drop(columns=['Unnamed: 0','product_material'])
df['image_link_color'] = df.apply(lambda x: get_dic_image(x), axis=1)
df = get_lowlevel(df)
df['gender'] = df['display_name'].apply(lambda x: change_gender(x))
df['scrapped_date'] = dt.date.today()

dresse?s?$
dresse?s?$
jeans
dresse?s?$
dresse?s?$
dresse?s?$
dresse?s?$
dresse?s?$
dresse?s?$
dresse?s?$
dresse?s?$
dresse?s?$
jeans
dresse?s?$
loafers?
tops?
dresse?s?$
dresse?s?$
jeans
pants?
tops?
two piece
dresse?s?$
tops?
dresse?s?$
blazers?
dresse?s?$
sk[io]rts?
dresse?s?$
dresse?s?$
dresse?s?
dresse?s?$
tops?
jeans
bustier
dresse?s?$
jeans
jeans
dresse?s?$
dresse?s?$
jeans
dresse?s?$
dresse?s?
dresse?s?$
two piece
dresse?s?$
dresse?s?$
sk[io]rts?
sk[io]rts?
dresse?s?
bra
sk[io]rts?
tops?
jeans
dresse?s?$
dresse?s?$
tops?
dresse?s?$
jeans
jackets?
jeans
tops?
sk[io]rts?
jackets?
dresse?s?$
dresse?s?
dresse?s?$
dresse?s?$
blazers?
pants?
dresse?s?$
tops?
dresse?s?$
tops?
dresse?s?
tops?
dresse?s?$
two piece
dresse?s?$
dresse?s?
dresse?s?$
sk[io]rts?
dresse?s?$
two piece
dresse?s?
dresse?s?$
jackets?
bra
two piece
tops?
dresse?s?$
dresse?s?$
dresse?s?$
dresse?s?$
two piece
dresse?s?$
dresse?s?$
pants?
dresse?s?$
sweaters?
jeans
tops?
jeans
tops?
tops?
sk[io]rts?
jeans
tops?
dresse?

In [1296]:
df[['display_name','low_level_new']]

,display_name,low_level_new
0,Bard Dress,Midi Dresses
1,Lloyd Dress,Midi Dresses
2,Liza Ultra High Rise Straight Jeans,Jeans
3,Fulton Dress,Midi Dresses
4,Karah Dress,Midi Dresses
5,Enid Dress,Midi Dresses
6,Gavin Dress,Midi Dresses
7,Alessi Dress,Midi Dresses
8,Murietta Dress,Midi Dresses
9,Benedict Dress,Midi Dresses


In [1297]:
df[df['low_level_new']=='other'][['display_name','low_level_new']]

,display_name,low_level_new
19,Abalonia Chunky Maryjane,other
80,Nashville Linen Short Es,other
117,Charlie High Rise Jean Short,other
126,Kayo High &amp; Skinny,other
193,Cashmere Sweatsuit,other
195,Cashmere Relaxed V-Neck,other


In [1298]:
df = df.rename(columns={"low_level_new": "low_level"})

In [1299]:
df = df.rename(columns={"id": "clothing_id"})

In [1300]:
new_df_materials = []
for i in range(len(df.materials)):
    material_dict={}
    for percentage_material in df['materials'].iloc[i]:
        percentmat_list = percentage_material.split('% ')
        for i in range(0,len(GranRegs.index)):
            if percentmat_list[1] in GranRegs.iloc[i].to_list():
                new_mat_id = GranRegs[0][i]
        #print(new_mat_id)
        material_dict.update({new_mat_id:percentmat_list[0] + "%"})
    new_df_materials.append(material_dict)
df.materials = pd.Series(new_df_materials)

In [1301]:
df.drop(df[df["materials"].str.len() == 0]['fabric'].index, inplace=True)

In [1303]:
df = df[df['low_level'] != 'other']

## Scoring

In [1304]:
import psycopg2
import json

In [1305]:
#conntect to aws database
connection = psycopg2.connect(
    host = 'changingroom-database1.cfiednof9fbl.us-east-2.rds.amazonaws.com',
    port = 5432,
    user = 'change_admin',
    password = 'ChangeFashion2020!',
    database='marketplace'
    )
cursor=connection.cursor()

# Create required materials dataframe
sql = """ SELECT * FROM materials """
higg = pd.read_sql(sql, con=connection)

# Create required brands dataframe
sql = """ SELECT * FROM goodonyou """
good = pd.read_sql(sql, con=connection)

# Create required items dataframe
sql = """ SELECT * FROM item """
item_df = pd.read_sql(sql, con=connection)

connection.close()

In [1306]:
def get_scores_material_dict(df):
    
    brand_name = df['brand_name']
    material_dict = df['materials']
    #print(material_dict)

    ################################################ INPUT ################################################


    # Material info
    materials = list(material_dict.keys())
    weights = list(material_dict.values())
    n = len(weights)
    
    

    ############################################ GETTING SCORES ###############################################
    # making sure the weights are tractable numbers
    for i in range(len(weights)):
        weights[i] = float(weights[i].replace('%', ''))/100.0
    
    
    # scores
    GlobalWarmingList = []
    EutrophicationList = []
    WaterScarcityList = []
    AbioticResourceDepletionList = []
    FiberTypeList = []
    MicroplasticPollutionList = []
    RegenerativeList=[]
    CO2WeightList = []
    WaterVolumeList = []


    # Metrics of CO2, water and Fossil Fuel usage
    CO2Weight = 0.0
    WaterVolume = 0.0
    RecyclabilityScore = 0.0
    RegenerativeFactor = 0.0

    #boolean values to calculate recyclability
    isRecyclableMat = True 
    isAllSynthetic = True 
    isBiodegradable = True 


    # looping over every material
    for i in range(len(materials)):

        row = higg[higg['material_id'] == str(materials[i])]
        #print(row)
        
        #higg score
        GlobalWarmingList.append(
            weights[i]*float(row['score_globwarm']))
        EutrophicationList.append(
            weights[i]*float(row['score_eutrophication']))
        WaterScarcityList.append(
            weights[i]*float(row['score_water']))
        AbioticResourceDepletionList.append(
            weights[i]*float(row['score_resourcedepletion']))

        #higg CO2, Water metrics
        CO2WeightList.append(weights[i]*float(row['co2e_kg']))
        WaterVolumeList.append(weights[i]*float(row['water_volume_m3']))


        #circularity score
        FiberTypeList.append(row['is_natural'].iloc[0])
        #print(row)
        if row['is_recyclable'].iloc[0] == 'Non-Recyclable':
            isRecyclableMat = False
        if row['is_biodegradable'].iloc[0] == 'Not Biodegradable':
            isBiodegradable = False
        if row['is_microplastic_pollutant'].iloc[0] == 'Microplastic Pollutant':
            MicroPollutantFactor = 1.0
        else:
            MicroPollutantFactor = 5.0
        MicroplasticPollutionList.append(weights[i]*MicroPollutantFactor)
        if row['is_regenerative'].iloc[0] == 'Non-Regenerative':
            RegenerativeFactor = 1.0
        elif row['is_regenerative'].iloc[0] == 'Semi-Regenerative':
            RegenerativeFactor = 3.0
        elif row['is_regenerative'].iloc[0] == 'Regenerative':
            RegenerativeFactor = 5.0
        RegenerativeList.append(weights[i]*RegenerativeFactor)



################################################ OUTPUT ################################################

    #HIGG SCORE compute the sum of weight*material_score
    WaterScore = 0.0
    EutrophiScore = 0.0
    CO2Score = 0.0
    FossilFuelScore = 0.0

    for ws in WaterScarcityList:
        WaterScore += ws
    for es in EutrophicationList:
        EutrophiScore += es
    for gs in GlobalWarmingList:
        CO2Score += gs
    for ard in AbioticResourceDepletionList:
        FossilFuelScore += ard

    #higg MSI impact metrics
    for footprint in CO2WeightList:
        CO2Weight += footprint
    for wate_wasted in WaterVolumeList:
        WaterVolume += wate_wasted


    #CIRCULARITY SCORES:

    #RECYCLABILITY
    if len(set(FiberTypeList)) != 1: #different fiber types equals non recyclability
        isAllSynthetic = False
        RecyclabilityScore = 1.0

    if isRecyclableMat is False:
        RecyclabilityScore = 1.0
    elif isRecyclableMat and isAllSynthetic is True:
        RecyclabilityScore = 5
    else:
        RecyclabilityScore = 1.0

    #BIODEGRADABLE?
    if isBiodegradable is True:
        BiodegradableScore = 5.0
    else:
        BiodegradableScore = 1.0

    #MICROPLASTIC POLLUTION
    MicroplasticPollutionScore = 0.0
    for mpp in MicroplasticPollutionList:
        MicroplasticPollutionScore += mpp

    #REGENERATIVENESS
    RegenerativeScore = 0.0
    for rg in RegenerativeList:
        RegenerativeScore += rg


    # GoodOnYou score
    good_score_loc = good[good['frontend_brandname'].str.contains(
        brand_name, case=False)]
    if good_score_loc.empty:
        good_score_loc = good[good['description'].str.contains(
            brand_name, case=False)]
        if good_score_loc.empty:
            planet_score = None
            people_score = None
        else:
            planete_sc = good_score_loc['rating'].loc[good_score_loc['domain'] == 'Planet']
            people_sc = good_score_loc['rating'].loc[good_score_loc['domain'] == 'People']
            planet_score = planete_sc.iloc[0][0] #extract the score from string
            people_score = people_sc.iloc[0][0] #extract the score from string
    else:
        if len(good_score_loc) == 3:
            planete_sc = good_score_loc['rating'].loc[good_score_loc['domain'] == 'Planet']
            people_sc = good_score_loc['rating'].loc[good_score_loc['domain'] == 'People']
            planet_score = planete_sc.iloc[0][0] #extract the score from string
            people_score = people_sc.iloc[0][0] #extract the score from string
        else:
            planet_score = None
            people_score = None
            pass


    water_score = round(WaterScore, 3)
    eutrophi_score = round(EutrophiScore, 3)
    co2_score = round(CO2Score, 3)
    fossilfuel_score = round(FossilFuelScore, 3)

    #TOTAL SCORE WEIGHTS
    brand_weight = 0.5
    higg_weight = 0.25
    circularity_weight = 0.25

    #number of components
    num_brand_score_component = 2
    num_higg_component = 4
    num_circularity_component = 4

    if planet_score is not None:
        brand_score = (float(planet_score) + float(people_score))/num_brand_score_component
        higg_score = (WaterScore + EutrophiScore + CO2Score + FossilFuelScore) / num_higg_component
        circularity_score = (RecyclabilityScore + BiodegradableScore + MicroplasticPollutionScore + RegenerativeScore)/num_circularity_component
        overall_score = higg_weight*higg_score + brand_weight*float(brand_score) + circularity_weight*circularity_score
        overall_score = round(overall_score, 3) 

    else:
        brand_score = None
        higg_score = (WaterScore + EutrophiScore + CO2Score + FossilFuelScore) / num_higg_component
        circularity_score = (RecyclabilityScore + BiodegradableScore + MicroplasticPollutionScore + RegenerativeScore)/num_circularity_component
        overall_score = (higg_weight*higg_score + brand_weight*float(brand_score))*2
        overall_score = round(overall_score, 3) 

    scores_table = pd.DataFrame([brand_name, water_score, eutrophi_score,
                        co2_score, fossilfuel_score, brand_score, planet_score, people_score, overall_score, 
                        circularity_score, higg_score, WaterVolume, CO2Weight,
                        RecyclabilityScore , BiodegradableScore , MicroplasticPollutionScore, RegenerativeScore]).transpose()

    scores_table.columns = ['BrandName', 'WaterScore', 'EutrophicationScore',
                            'CO2Score', 'FossilFuelScore', 'GoodOnYou', 'PlanetScore', 'PeopleScore', 'OverallScore', 
                            'CircularityScore', 'HiggScore','WaterVolume', 'CO2Weight',
                            'RecyclabilityScore', 'BiodegradableScore', 'MicroplasticPollutionScore', 'RegenerativeScore']

    return scores_table['OverallScore']
        

In [1307]:
df['OverallScore'] = df.apply(get_scores_material_dict,axis=1)

/Users/Jeremy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [1308]:
df['description'] = df['description'] + df['product_detail'] 

/Users/Jeremy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [1309]:
df.drop(columns=['fabric','sustainability', 'product_detail', 'data_analytics'], inplace=True)

/Users/Jeremy/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [1310]:
df = df[['clothing_id', 'display_name', 'color', 'size', 'price', 'product_url', 'image_link_color', 'brand_name', 
         'description', 'scrapped_date', 'low_level', 'materials', 'gender', 'OverallScore']]

In [1311]:
df['secondhand'] = 'No'

In [1312]:
df.rename({'materials' : 'material_dict'}, axis=1, inplace=True)

#### Push to DB

In [1314]:
df['color'] = df['color'].apply(lambda x: str(x))

In [1315]:
def formatDFforDB(df):
    
    
    clothing_df_copy = df.copy()
    clothing_df_copy['image_link_color'] = clothing_df_copy['image_link_color'].apply(json.dumps)
    clothing_df_copy['material_dict'] = clothing_df_copy['material_dict'].apply(json.dumps)
    clothing_df_copy['color'] = clothing_df_copy['color'].str.replace('[','{').str.replace(']','}')
    clothing_df_copy['size'] = clothing_df_copy['size'].str.replace('[','{').str.replace(']','}')
    clothing_df_copy['size'] = clothing_df_copy['size'].str.replace('"','inches')

    return clothing_df_copy

In [1316]:
df['image_link_color'].apply(json.dumps).iloc[0]

'{"Belvedere": "https://media.thereformation.com/image/upload/f_auto,q_auto:eco/c_scale,w_auto:breakpoints_100_2560_9_20/v1/prod/product_images/bard-dress/belvedere/612944e235e771001742b27b/original.jpg"}'

In [1317]:
formatDFforDB(df).to_csv('reformation-cleaned.csv',index=None)

In [11]:
pd.read_csv('tradesy_df_DB.csv')

,clothing_id,display_name,color,size,price,product_url,image_link_color,brand_name,description,scrapped_date,low_level,material_dict,gender,overallscore,secondhand
0,20210825Tradesy0,Medium Red Leather Shoulder Bag,{'Red'},{'15inchesL x 0.5inchesW x 9.5inchesH'},$29.80,https://www.tradesy.com/i/via-spiga-medium-red...,"""\""{'Red': 'https://img-static.tradesy.com/ite...",Tradesy,via spiga red medium shoulder bag\r\nhandle dr...,2021-07-29,Shoulder Bag,"""NaN""",Women,4.5,Yes
1,20210825Tradesy1,Black/Silver and Black Beading Halter Top Form...,{'Black'},{'Size: 4 (S) Length: Long'},$77.27,https://www.tradesy.com/i/jordan-fashions-blac...,"""\""{'Black': 'https://img-static.tradesy.com/i...",Tradesy,beautiful new dress (i'm closing my store) sel...,2021-07-29,Formal Dresses,"""NaN""",Women,4.5,Yes
2,20210825Tradesy2,Pewter Metallic Jeweled Sandals,{'Unknown'},"{'Size: US 7 Regular (M, B)'}",$41.80,https://www.tradesy.com/i/bandolino-pewter-met...,"""\""{'Unknown': 'https://img-static.tradesy.com...",Tradesy,name: the zarba sandal\r\nstyle #: 0809\r\ndes...,2021-07-29,Sandals,"""NaN""",Women,4.5,Yes
3,20210825Tradesy3,Black Zip Front Peep Toe Sandal Heel Boots/Boo...,{'Black'},"{'Size: US 7 Regular (M, B)'}",$662.80,https://www.tradesy.com/i/givenchy-black-zip-f...,"""\""{'Black': 'https://img-static.tradesy.com/i...",Tradesy,new givenchy zip front peep toe and sandal hee...,2021-07-29,Boots,"""NaN""",Women,4.5,Yes
4,20210825Tradesy4,Leopard Peep Pumps,{'Purple'},"{'Size: US 8.5 Regular (M, B)'}",$162.50,https://www.tradesy.com/i/l-autre-chose-leopar...,"""\""{'Purple': 'https://img-static.tradesy.com/...",Tradesy,leopard peep toe pumps\r\nstyle: l6905cav\r\nd...,2021-07-29,Heels,"""NaN""",Women,4.5,Yes
5,20210825Tradesy5,Tropical Blue Jersey Halter Night Out Dress,{'Blue'},{'Size: 4 (S) Length: Above Knee'},$77.36,https://www.tradesy.com/i/halston-tropical-blu...,"""\""{'Blue': 'https://img-static.tradesy.com/it...",Tradesy,"describe the fit, condition, etc.",2021-07-29,Night Out Dresses,"""NaN""",Women,4.5,Yes
6,20210825Tradesy6,Black Sequence Skirt Cocktail Dress,{'Black'},{'Size: 0 (XS) Length: Mini'},$24.00,https://www.tradesy.com/i/bcbgmaxazria-black-s...,"""\""{'Black': 'https://img-static.tradesy.com/i...",Tradesy,NaN,2021-07-29,Skirts,"""NaN""",Women,4.5,Yes
7,20210825Tradesy7,Green Asian Style Night Out Dress,{'Green'},{'Size: 4 (S) Length: Above Knee'},$50.00,https://www.tradesy.com/i/akiko-green-asian-st...,"""\""{'Green': 'https://img-static.tradesy.com/i...",Tradesy,NaN,2021-07-29,Night Out Dresses,"""NaN""",Women,4.5,Yes
8,20210825Tradesy8,Skin with Brass & Leather Details Brown and Wh...,{'White'},{'13inchesL x 3inchesW x 8.5inchesH'},$554.85,https://www.tradesy.com/i/dolce-and-gabbana-sk...,"""\""{'White': 'https://img-static.tradesy.com/i...",Tradesy,dolce & gabbana cow skin purse with brass and ...,2021-07-29,Bra,"""NaN""",Women,4.5,Yes
9,20210825Tradesy9,Camo Sling Back Pumps,{'Unknown'},"{'Size: US 8 Regular (M, B)'}",$85.20,https://www.tradesy.com/i/rock-and-republic-ca...,"""\""{'Unknown': 'https://img-static.tradesy.com...",Tradesy,great rock & republic camo sling back pumps\r\...,2021-07-29,Heels,"""NaN""",Women,4.5,Yes


In [3]:
#conntect to aws database
connection = psycopg2.connect(
    host = 'changingroom-database1.cfiednof9fbl.us-east-2.rds.amazonaws.com',
    port = 5432,
    user = 'change_admin',
    password = 'ChangeFashion2020!',
    database='marketplace'
    )
cursor=connection.cursor()


In [4]:
#here is an example to create table materials with columns material_id of type text, chemistry of type float etc.
#each row of a table should ideally be uniquely identifiable by an id so always make sure to have an id column
cursor.execute("""CREATE TEMP TABLE tradesy(
clothing_id text PRIMARY KEY,
display_name text,
color text ARRAY[20],
size text ARRAY[20],
price text,
product_url text,
image_link_color JSONB,
brand_name text,
description text,
scrapped_date date,
low_level text,
material_dict JSONB,
gender text,
overallscore float,
secondhand text);""")

connection.commit()

In [7]:
### UPDATE NEW DATA

#option 1 - when the csv is in the exact form/schema of the table to upload
csv_file_name = 'tradesylow.csv'
sql = "COPY tradesy FROM STDIN DELIMITER ',' CSV HEADER"
cursor.copy_expert(sql, open(csv_file_name, "r"))

connection.commit()

InvalidTextRepresentation: malformed array literal: "["'Green'"]"
DETAIL:  "[" must introduce explicitly-specified array dimensions.
CONTEXT:  COPY tradesy, line 2, column color: "["'Green'"]"


In [8]:
connection.rollback()

In [1321]:
#here is an example to create table materials with columns material_id of type text, chemistry of type float etc.
#each row of a table should ideally be uniquely identifiable by an id so always make sure to have an id column
cursor.execute("""
UPDATE clothes
SET    overallscore = tradesy.overallscore
FROM   tradesy
WHERE  clothes.clothing_id = tradesy.clothing_id;

DROP TABLE reformation;""")

connection.commit()

In [1322]:
connection.close()

In [866]:
df.OverallScore

0      3.707
1      3.454
2      2.741
3      3.707
4      2.860
5      3.266
6      3.266
7      3.266
8      3.707
9      3.536
10     3.726
11     3.707
12     3.308
13     3.707
14     3.469
15     3.726
16     3.707
17     3.707
18     2.741
19     3.469
20     3.283
21     3.223
22     3.726
23     3.707
24     3.438
25     3.707
26     3.469
27     3.707
28     3.266
29     3.266
       ...  
168    3.959
169    3.959
170    3.959
171    3.428
172    2.725
173    3.959
174    3.889
175    3.959
176    3.959
177    3.959
178    3.959
179    3.959
180    3.959
181    3.959
182    3.959
183    3.200
184    3.959
185    3.959
186    3.200
187    3.959
188    3.959
189    3.889
190    3.959
191    3.889
192    3.959
193    3.200
194    3.200
195    3.200
196    3.889
197    3.130
Name: OverallScore, Length: 194, dtype: float64

In [912]:
df

,clothing_id,display_name,color,size,price,product_url,image_link_color,brand_name,description,scrapped_date,low_level,material_dict,gender,OverallScore,secondhand
0,1309357,Bard Dress,['Belvedere'],"{'0': 'backordered', '2': 'backordered', '4': ...",218.0,https://www.thereformation.com/products/bard-d...,"{""Belvedere"": ""https://media.thereformation.co...",Reformation,In case of getting dressed. The Bard gives you...,2021-09-15,Midi Dresses,"{""tex_visc_36"": ""100%""}",Women,3.707,No
1,1309275,Lloyd Dress,"['Pistachio', 'Black']","{'XS': 'available', 'S': 'available', 'M': 'av...",278.0,https://www.thereformation.com/products/lloyd-...,"{""Black"": ""https://media.thereformation.com/im...",Reformation,All your other dresses are gonna be jealous. T...,2021-09-15,Midi Dresses,"{""tex_cott_7"": ""98%"", ""tex_elas_10"": ""2%""}",Women,3.454,No
2,1306151,Liza Ultra High Rise Straight Jeans,"['Malta', 'Black', 'Anguilla', 'Aden']","{'23': 'available', '24': 'available', '25': '...",118.0,https://www.thereformation.com/products/stevie...,"{""Aden"": ""https://media.thereformation.com/ima...",Reformation,Nice legs. The Liza Ultra is slim fitting thro...,2021-09-15,Tank Tops,"{""tex_cott_7"": ""33%"", ""tex_elas_10"": ""1%""}",Women,2.741,No
3,1305653,Fulton Dress,"['Princess Margaret', 'Tuli', 'Jaspar']","{'0': 'available', '2': 'available', '4': 'ava...",248.0,https://www.thereformation.com/products/fulton...,"{""Jaspar"": ""https://media.thereformation.com/i...",Reformation,Let those legs out for the day.\tThe Fulton is...,2021-09-15,Skirts,"{""tex_visc_36"": ""100%""}",Women,3.707,No
4,1309270,Karah Dress,['Cream'],"{'XS': 'available', 'S': 'available', 'M': 'av...",128.0,https://www.thereformation.com/products/karah-...,"{""Cream"": ""https://media.thereformation.com/im...",Reformation,"Who, me. The Karah has a fitted bust with an e...",2021-09-15,other,"{""tex_poly_25"": ""22%"", ""tex_elas_10"": ""4%""}",Women,2.860,No
5,1309354,Enid Dress,['Autumnal'],"{'0': 'backordered', '2': 'backordered', '4': ...",248.0,https://www.thereformation.com/products/enid-d...,"{""Autumnal"": ""https://media.thereformation.com...",Reformation,Wear the dress. The Enid has a slim fitting bo...,2021-09-15,Midi Dresses,"{""tex_visc_36"": ""47%""}",Women,3.266,No
6,1303035,Gavin Dress,"['Emerald', 'Black', 'Teacup', 'Flower Girl', ...","{'0': 'available', '2': 'available', '4': 'ava...",218.0,https://www.thereformation.com/products/gavin-...,"{""Cheetah"": ""https://media.thereformation.com/...",Reformation,Not too mini or maxi. This is a midi length dr...,2021-09-15,Skirts,"{""tex_visc_36"": ""47%""}",Women,3.266,No
7,1307222,Alessi Dress,"['Summer Soiree', 'Cheetah']","{'0': 'available', '2': 'available', '4': 'ava...",248.0,https://www.thereformation.com/products/alessi...,"{""Cheetah"": ""https://media.thereformation.com/...",Reformation,House specialty. The Alessi is fitted in the b...,2021-09-15,Skirts,"{""tex_visc_36"": ""47%""}",Women,3.266,No
8,1309358,Murietta Dress,['Tapioca'],"{'XS': 'backordered', 'S': 'backordered', 'M':...",248.0,https://www.thereformation.com/products/muriet...,"{""Tapioca"": ""https://media.thereformation.com/...",Reformation,What do we have here? The Murietta is a mini w...,2021-09-15,Brimmed Hats,"{""tex_visc_36"": ""100%""}",Women,3.707,No
9,1309271,Benedict Dress,['Chestnut'],"{'XS': 'available', 'S': 'available', 'M': 'av...",148.0,https://www.thereformation.com/products/benedi...,"{""Chestnut"": ""https://media.thereformation.com...",Reformation,You must be new. The Benedict has a slim fit t...,2021-09-15,Skirts,"{""tex_poly_25"": ""94%"", ""tex_elas_10"": ""6%""}",Women,3.536,No
